# Data Science Assignment 1
Kishan Ved

22110122

In [271]:
import pandas as pd
import numpy as np

df = pd.read_csv('musae_facebook_target.csv');

In [272]:
df

,id,facebook_id,page_name,page_type
0,0,145647315578475,The Voice of China 中国好声音,tvshow
1,1,191483281412,U.S. Consulate General Mumbai,government
2,2,144761358898518,ESET,company
3,3,568700043198473,Consulate General of Switzerland in Montreal,government
4,4,1408935539376139,Mark Bailey MP - Labor for Miller,politician
...,...,...,...,...
22465,22465,1379955382222841,Kurt Wiegel MdL,politician
22466,22466,1651527995097082,dubdub Stories,company
22467,22467,155369444540412,Ministerio del Interior - Paraguay,government
22468,22468,175067819212798,Tottus Perú,company


In [273]:
edges_df = pd.read_csv("musae_facebook_edges.csv")

In [274]:
edges_df

,id_1,id_2
0,0,18427
1,1,21708
2,1,22208
3,1,22171
4,1,6829
...,...,...
170997,20188,20188
170998,22340,22383
170999,22348,22348
171000,5563,5563


### Road map

1. tot - deg[v] = e(s,s)  {deg is a dict containing degrees}

2. Remove v from deg

3. Update deg of vertices that were connected to v

4. Find the density 

5. Store the vertex removed and the density of the subgraph obtained after this

In [276]:
deg = {}

for _, row in edges_df.iterrows():
    for node in ['id_1', 'id_2']:
        if row[node] in deg:
            deg[row[node]] += 1
        else:
            deg[row[node]] = 1

In [277]:
deg # dictionary of degrees

{0: 1,
 18427: 51,
 1: 34,
 21708: 195,
 22208: 205,
 22171: 330,
 6829: 27,
 16590: 87,
 20135: 140,
 8894: 52,
 15785: 147,
 10281: 13,
 22265: 226,
 7136: 25,
 22405: 41,
 10379: 468,
 13737: 62,
 8533: 36,
 14344: 50,
 2812: 45,
 5755: 68,
 16260: 26,
 15026: 42,
 17370: 193,
 17460: 66,
 8049: 193,
 5307: 122,
 4987: 99,
 18304: 25,
 12305: 67,
 19743: 678,
 20024: 128,
 21729: 659,
 10554: 56,
 11557: 72,
 5228: 64,
 9934: 71,
 2: 12,
 9048: 14,
 6353: 23,
 2629: 2,
 11537: 22,
 13205: 20,
 22304: 20,
 17728: 9,
 19337: 14,
 126: 23,
 17554: 11,
 8495: 17,
 5857: 11,
 3: 10,
 16742: 21,
 293: 59,
 5826: 36,
 3479: 22,
 19753: 32,
 17346: 15,
 10945: 46,
 22338: 46,
 11319: 14,
 9654: 8,
 4: 51,
 13645: 65,
 20876: 32,
 11446: 22,
 16203: 56,
 2830: 20,
 2004: 37,
 20624: 38,
 21280: 47,
 1182: 32,
 21538: 26,
 1443: 71,
 11423: 29,
 187: 142,
 5730: 16,
 2983: 35,
 1489: 22,
 6823: 43,
 17695: 112,
 11102: 158,
 6390: 64,
 17242: 20,
 10018: 32,
 5147: 31,
 6427: 41,
 14628: 70,


In [278]:
tot = sum(deg.values())/2
tot # e(S,S)

171002.0

In [279]:
ctr = len(deg) # |S|
ctr

22470

In [280]:
total_vertices = len(deg)

In [281]:
density = [] # stores densities
vertex = [] # vertex[0:i] (i included) contains vertices removed to achive density[i]

In [282]:
def find_density_greedy():
    global tot
    global deg
    global ctr
    global edges_df
    global vertex
    
    v = min(deg, key=lambda k: deg[k]) # vertex with minimum degree
    vertex.append(v) # Store the removed vertex
    ctr = ctr - 1 # decrease |S| by 1 (as v is removed)
    tot = tot - deg[v] # Update e(S,S)
    den = tot / ctr
    density.append(den)


    selected_rows = edges_df[edges_df['id_1'] == v]
    
    deg_update = selected_rows['id_2'].value_counts().to_dict()
    for ver, count in deg_update.items():
        if ver in deg:
            deg[ver] -= count
    deg.pop(v, None) # Remove v from deg, None for no error if v is not in deg

    edges_df = edges_df[~edges_df.isin([v]).any(axis=1)]
    
    



In [283]:
i = 1
while(ctr>1):
    i+=1
    if (i%1000==0):
        print(i)
    find_density_greedy()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [284]:
density = np.array(density)

In [285]:
np.argmax(density)

17132

In [286]:
x = np.argmax(density)

In [298]:
density[x]

15.130222971706951

In [287]:
graph = []
for i in range(0, total_vertices):
    if i not in vertex[:x+1]:
        graph.append(i)
    

In [289]:
len(graph)

5337

In [297]:
dict = {}
for ver in graph:
    type = df.loc[ver, 'page_type']
    if(type in dict.keys()):
        dict[type] += 1
    else:
        dict[type] = 1

dict

{'government': 3076, 'politician': 1429, 'tvshow': 388, 'company': 444}